In [40]:
# modules

import csv
from collections import OrderedDict
from collections import Counter
import requests
import sqlite3
import io
import os

In [106]:
# definitions, datasets

def in_list(query, item_list, any_all):
    if any_all == 'any':
        return any(query in item for item in item_list)
    if any_all == 'all':
        return all(query in item for item in item_list)

def faculty_data(dept, faculties, author = ''):
    export = []
    for faculty in faculties:
        if dept in faculty['Dept']:
            export.append(faculty['Last English'].lower())
    if author == '':
        return export
    else:
        return [in_list(author.lower(), export, 'any'), ]

def dict_factory(cursor, row):
    key_map = {
        'Authors': 'auts',
        'Author(s) ID': 'auts_id',
        'Title': 'title',
        'Year': 'year',
        'Source title': 'src',
        'Volume': 'vol',
        'Issue': 'issue',
        'Art. No.': 'art_no',
        'Page start': 'pg_start',
        'Page end': 'pg_end',
        'Page count': 'pg_count',
        'Cited by': 'cites',
        'DOI': 'doi',
        'Link': 'link',
        'Affiliations': 'affils',
        'Authors with affiliations': 'auts_affils',
        'Abstract': 'abs',
        'Author Keywords': 'aut_key',
        'Index Keywords': 'ind_key',
        'Molecular Sequence Numbers': 'molecular_sequence_numbers',
        'Chemicals/CAS': 'cas',
        'Tradenames': 'tradenames',
        'Manufacturers': 'manufacturers',
        'Funding Details': 'fund',
        'References': 'refs',
        'Correspondence Address': 'address',
        'Editors': 'editors',
        'Sponsors': 'sponsors',
        'Publisher': 'publisher',
        'Conference name': 'conf_name',
        'Conference date': 'conf_date',
        'Conference location': 'conf_loc',
        'Conference code': 'conf_code',
        'ISSN': 'issn',
        'ISBN': 'isbn',
        'CODEN': 'coden',
        'PubMed ID': 'pubmed',
        'Language of Original Document': 'lang',
        'Abbreviated Source Title': 'abb_src',
        'Document Type': 'doc_type',
        'Publication Stage': 'pub_stage',
        'Access Type': 'access_type',
        'Source': 'source',
        'EID': 'eid',
    }
    dic = OrderedDict()
    if key_map:
        for idx, col in enumerate(cursor.description):
            dic[key_map[col[0]]] = row[idx]
    else:
        for idx, col in enumerate(cursor.description):
            dic[col[0]] = row[idx]
    return dic

usa = ['united states']

selected_countries = {
    'france': 'France', 
    'germany': 'Germany', 
    'turkey': 'Turkey', 
    'russia': 'Russia', 
    'china': 'China', 
    'united states': 'USA',
    'usa': 'USA',
}

sharif_depts = {
    'Department of Electrical Engineering': 'EE',
    'Electrical Engineering Department': 'EE',
    'Electrical Engineering': 'EE',
    
    'Department of Aerospace Engineering': 'Aero',
    'Aerospace Engineering Department': 'Aero',
    'Aerospace Engineering': 'Aero',
    
    'Department of Computer Engineering': 'CE',
    'Computer Engineering Department': 'CE',
    'Computer Engineering': 'CE',
    
    'Department of Industrial Engineering': 'IE',
    'Industrial Engineering Department': 'IE',
    'Industrial Engineering': 'IE',
    
    'Department of Energy Engineering': 'Energy',
    'Energy Engineering Department': 'Energy',
    'Energy Engineering': 'Energy',
    
    'Department of Mechanical Engineering': 'Mech',
    'Mechanical Engineering Department': 'Mech',
    'Mechanical Engineering': 'Mech',
    
    'Department of Civil Engineering': 'Civil',
    'Civil Engineering Department': 'Civil',
    'Civil Engineering': 'Civil',
    
    'Graduate School of Management and Economics': 'GSME',
    'Graduate School of Management & Economics': 'GSME',
    'Management and Economics': 'GSME',
    'Management & Economics': 'GSME',
    
    'Department of Materials Science and Engineering': 'MSE',
    'Department of Materials Science & Engineering': 'MSE',
    'Materials Science and Engineering Department': 'MSE',
    'Materials Science & Engineering Department': 'MSE',
    'Materials Science and Engineering': 'MSE',
    'Materials Science & Engineering': 'MSE',
    
    'Department of Chemical and Petroleum Engineering': 'ChE',
    'Department of Chemical & Petroleum Engineering': 'ChE',
    'Chemical and Petroleum Engineering Department': 'ChE',
    'Chemical & Petroleum Engineering Department': 'ChE',
    'Chemical and Petroleum Engineering': 'ChE',
    'Chemical & Petroleum Engineering': 'ChE',
    
    'Department of Chemistry': 'Chem',
    'Chemistry': 'Chem',
    'Department of Physics': 'Phys',
    'Physics': 'Phys',
    'Department of Mathematical Sciences': 'Math',
    'Mathematical': 'Math',
    
    # 'Institute for Nanoscience and Nanotechnology': 'Nano',
    # 'Institute for Nanoscience & Nanotechnology': 'Nano',
    # 'Nanoscience and Nanotechnology Institute': 'Nano',
    # 'Nanoscience & Nanotechnology Institute': 'Nano',
}


dataset_extention = ['txt', 'csv']
dataset_directory = 'datasets'

datasets = {}

for file in os.listdir(dataset_directory):
    if file.split('.')[1] in dataset_extention:
        db_name = file.split('.')[0]
        if db_name == 'Countries':
            datasets[db_name.lower()] = {}
            with open(os.path.join(dataset_directory, file), encoding='UTF-16') as import_file:
                reader = csv.DictReader(import_file, dialect='excel-tab')
                for row in reader:
                    datasets[db_name.lower()][row['Countries'].lower()] = {
                        'id': row['ID'],
                        'islamic': True if row['Islamic'] == 'True' else False,
                    }
        elif db_name == 'Faculties':
            datasets[db_name.lower()] = {}
            with open(os.path.join(dataset_directory, file), encoding='UTF-8-sig') as import_file:
                reader = csv.DictReader(import_file)
                for row in reader:
                    datasets[db_name.lower()].append(row)
        else:
            datasets[db_name.lower()] = []
            with open(os.path.join(dataset_directory, file), encoding='UTF-16') as import_file:
                reader = csv.DictReader(import_file, dialect='excel-tab')
                for row in reader:
                    datasets[db_name.lower()].append(row[db_name].lower())

In [ ]:
{
    'Aero': ['Aerospace Engineering'],
    'CE': ['Computer Engineering'],
    'ChE': ['Chemical and Petroleum Engineering', 'Chemical & Petroleum Engineering'],
    'Chem': ['Chemistry'],
    'Civil': ['Civil Engineering'],
    'EE': ['Electrical Engineering'],
    'Energy': ['Energy Engineering'],
    'GSME': ['Management and Economics', 'Management & Economics'],
    'IE': ['Industrial Engineering'],
    'MSE': ['Materials Science and Engineering', 'Materials Science & Engineering'],
    'Math': ['Mathematical'],
    'Mech': ['Mechanical Engineering'],
    'Nano': ['Nanoscience & Nanotechnology'],
    'Phys': ['Physics'],
}

In [95]:
# obtaining & parsing scopus data

# import_path = "papers\\2017.txt"


# papers = []
# with open(import_path, 'r', encoding='UTF-16') as tsvfile:
#     reader = csv.DictReader(tsvfile, dialect='excel-tab')
#     for row in reader:
#         papers.append(row)

query = '' or 'Department of Materials Science and Engineering, Sharif University of Technology'

db_query_aut = 'simchi'
db_query_year1 = '' #or 2016
db_query_year2 = 2018

db_name = 'papers.db'
papers_extension = 'csv'
papers_directory = 'papers'

db = '' or sqlite3.connect(db_name)

if type(db) == sqlite3.Connection:
    db.row_factory = dict_factory
    cursor = db.cursor()
    if db_query_year1 == '':
        cursor.execute(
            '''
                SELECT * FROM papers WHERE 
                Authors LIKE ?
            ''', 
            (
                '%' + db_query + '%',
            )
        )
    else:
        cursor.execute(
            '''
                SELECT * FROM papers WHERE 
                Year >= cast(? as numeric) AND 
                Year <= cast(? as numeric) AND 
                Authors LIKE ?
            ''', 
            (
                db_query_year1, db_query_year2, '%' + db_query + '%',
            )
        )
    papers = cursor.fetchall()

counter = 0
for i in papers:
    
    i['raw_countries'] = [] # gets the country name from the last piece in the address
    i['countries'] = [] # gets the country name, if it is in "selected_countries" or "islamic_countries"
    i['depts'] = []
    i['institutions'] = []
    i['sharif_only'] = True
    i['has_foreign'] = False
    i['has_islamic'] = False
    i['has_qs100'] = False
    i['has_qs300'] = False
    i['errors'] = {
        'country': {'type': 'Error', 'value': False},
        'qs': {'type': 'Error', 'value': False},
        'author': {'type': 'Warning', 'value': False, 'message': ''}
    }
    
    temp = OrderedDict()
    affils = i['affils'].split(';')
    for cnt, affil in enumerate(affils):
        temp[cnt] = affil.strip()
    i['affils'] = temp
    
    temp = OrderedDict()
    authors = i['auts'].split(',')
    for cnt, author in enumerate(authors):
        temp[cnt] = author.strip()
        
        # spliting with ('.') sometimes can cause problems. For example in J. Smith, III 
        if '.' not in author:
            print(f'Possible invalid author:\t{temp[cnt]}\ton\t{counter}')
            i['errors']['author']['value'] = True
            i['errors']['author']['message'] = f'Possible invalid author: {temp[cnt]}, on row {counter}'
    i['auts'] = temp
    
    authors_with_affil = i['auts_affils'].split(';')
    temp = OrderedDict()
    for cnt, author in enumerate(authors_with_affil):
        temp[cnt] = {
            'name_address': author.strip(),
            'sharif': False,
            'faculty': False,
            'dept': '',
            'foreign': False,
            'islamic': False,
            'country': '',
            'qs100': False,
            'qs300': False,
            'query': False,
        }
        # is the author affiliated with Sharif?
        # if 'sharif' in author.lower():
        if in_list('sharif', author.lower().split(',')[2:], 'any'):
            temp[cnt]['sharif'] = True
            i['institutions'].append('Sharif University of Technology')
            
            # if Author is from Sharif, what is his/her department?
            for element in author.split(','):
                if 'department' in element.lower() or 'institute' in element.lower():
                    element = element.strip()
                    # temp[cnt]['dept'] = element
                    if element in sharif_depts.keys():
                        temp[cnt]['dept'] = sharif_depts[element]
                        temp[cnt]['faculty'] = faculty_data(temp[cnt]['dept'], datasets['faculties'], author.split(',')[0])
                        # if temp[cnt]['dept'] not in datasets['faculties'][339]['Dept'] and :
                        #     print(temp[cnt]['name_address'])
                        #     print('******************')
                        #     print('ERRRRR')
                        #     print(counter)
                        #     print(element)
                        #     print(i['doi'])
                        #     print('-/-/-/-/-/-/-/-')
                        i['depts'].append(temp[cnt]['dept'])
                        break
        else:
            i['sharif_only'] = False
            i['institutions'].append(', '.join(map(lambda element: element.strip(), author.split(',')[2:-2])))
        
        # is Author from Iran?
        if not(in_list('iran', author.lower().split(',')[2:], 'any')):
            temp[cnt]['foreign'] = True
            i['has_foreign'] = True
            
            for country in datasets['countries']:
                if country in author.lower():
                    temp[cnt]['country'] = datasets['countries'][country]['id']
                    if datasets['countries'][country]['islamic']:
                        temp[cnt]['islamic'] = True
                        i['has_islamic'] = True
                    break
        else:
            temp[cnt]['country'] = 'IR'
        i['countries'].append(temp[cnt]['country'])
        i['raw_countries'].append(author.split(',')[-1].strip())
        
        # check whether the author is affiliated with any of the top 100 or top 101-300 universities in QS ranking
        if any(uni.lower() in author.lower() for uni in datasets['qs100']):
            temp[cnt]['qs100'] = True
            i['has_qs100'] = True
            
        if any(uni.lower() in author.lower() for uni in datasets['qs300']):
            temp[cnt]['qs300'] = True
            i['has_qs300'] = True
        if temp[cnt]['qs100'] and temp[cnt]['qs300']:
            i['errors']['qs']['value'] = True
            # print(f'QS Error on row: {counter}')

        
        if query.lower() in author.lower():
            temp[cnt]['query'] = True
    
    i['auts_affils'] = temp
    
    # is the paper journal from JCR top 1% or top 25%?
    i['src'] = OrderedDict({
        'journal': i['src'], 
        'jcr_q1': False, 
        'jcr_top1': False, 
        'a_hci': False, 
        'esci': False, 
        'scie': False, 
        'ssci': False, 
    })
    if i['src']['journal'].lower() in datasets['jcr'][:130]:
        i['src']['jcr_top1'] = True
        i['src']['jcr_q1'] = True
    if i['src']['journal'].lower() in datasets['jcr']:
        i['src']['jcr_q1'] = True
    
    if i['src']['journal'].lower() in datasets['a_hci']:
        i['src']['a_hci'] = True
    if i['src']['journal'].lower() in datasets['esci']:
        i['src']['esci'] = True
    if i['src']['journal'].lower() in datasets['scie']:
        i['src']['scie'] = True
    if i['src']['journal'].lower() in datasets['ssci']:
        i['src']['ssci'] = True
    counter += 1
    # print(counter, i['year'])
    # print(i['auts'])
    # print('---------------------------')

# printing out some preliminary results to see if everything runs OK
QS100 = []
QS300 = []
JCRQ1 = []
JCRTop1 = []
errors = []
warnings = []
for i in papers:
    if any(i['auts_affils'][cnt]['qs100'] == True for cnt in i['auts_affils']):
        QS100.append(i)
    if any(i['auts_affils'][cnt]['qs300'] == True for cnt in i['auts_affils']):
        QS300.append(i)

    if i['src']['jcr_q1'] == True:
        JCRQ1.append(i)
    if i['src']['jcr_top1'] == True:
        JCRTop1.append(i)
    
    if any(i['errors'][cnt]['value'] == True and i['errors'][cnt]['type'] == 'Error' for cnt in i['errors']):
        errors.append(i)
    if any(i['errors'][cnt]['value'] == True and i['errors'][cnt]['type'] == 'Warning' for cnt in i['errors']):
        warnings.append(i)

print('--------------------------------------------------')
print('QS100: ', len(QS100), '*** %: ', round(100 * len(QS100)/len(papers), 1), '*** Total: ', len(papers))
print('QS300: ', len(QS300), '*** %: ', round(100 * len(QS300)/len(papers), 1), '*** Total: ', len(papers))
print(f'Errors: {len(errors)} *** Warnings: {len(warnings)}')
# papers[105]

if type(db) == sqlite3.Connection:
    db.close()

Vossoughi, M., Institute for Nanoscience and Nanotechnology (INST), Sharif University of Technology, Azadi Ave., 14588 Tehran, Iran, Department of Chemical and Petroleum Engineering, Sharif University of Technology, Azadi Ave., 14588 Tehran, Iran
******************
ERRRRR
45
Department of Chemical and Petroleum Engineering
10.1016/j.msec.2011.06.013
-/-/-/-/-/-/-/-
Vossoughi, M., Institute for Nanoscience and Nanotechnology (INST), Sharif University of Technology, Azadi Ave., 14588 Tehran, Iran, Department of Chemical and Petroleum Engineering, Sharif University of Technology, Azadi Ave., 14588 Tehran, Iran
******************
ERRRRR
48
Department of Chemical and Petroleum Engineering
10.1016/j.matlet.2011.05.037
-/-/-/-/-/-/-/-
Shidpour, R., Institute for Nanoscience and Nanotechnology, Sharif University of Technology, Tehran, Iran, Department of Chemistry, University of California, Riverside, Riverside, CA, 92521, United States
******************
ERRRRR
94
Department of Chemistry
10.1

In [ ]:
def airtable(data, method = 'POST'):
    airtable_apikey = ''
    airtable_base   = 'https://api.airtable.com/v0/appP1aQMyImJ2UhAM/'
    airtable_table  = 'Papers'
    headers = {
        'Authorization': 'Bearer ' + airtable_apikey,
        'Content-type': 'application/json'
    }
    params = {
        'view': 'Grid view'
    }
    if method == 'POST':
        return requests.post(airtable_base + airtable_table, headers = headers, json = data).json()

In [ ]:
airtable_data = {
    'fields': {},
    'typecast': True,
}
foreign = []
islamic = []
for i in papers:
    if any(i['auts_affils'][j]['foreign'] == True for j in i['auts_affils']):
        foreign.append(i)
#     if (any(i['auts_affils'][j]['foreign'] == True for j in i['auts_affils']) and 
#         any(i['auts_affils'][j]['query'] == True for j in i['auts_affils'])):
        
#         airtable_data['fields']['doi'] = 'dx.doi.org/' + i['doi']
#         airtable_data['fields']['year'] = i['year']
#         airtable_data['fields']['title'] = i['title']
#         airtable_data['fields']['Sharif Authors'] = [i['auts'][j] for j in i['auts'] if i['auts_affils'][j]['Sharif'] == True]
#         airtable_data['fields']['Foreign Authors'] = [i['auts'][j] for j in i['auts'] if i['auts_affils'][j]['foreign'] == True]
#         airtable_data['fields']['Sharif Depts'] = list(set(i['Depts']))
        
#         airtable_data['fields']['Source']    = i['Src']['Journal']
#         airtable_data['fields']['Countries'] = [country for country in i['countries'] if country != 'Iran']
#         airtable_data['fields']['Query']     = 'Dept: MSE & Foreign: True'
        
# #         airtable(airtable_data, method = 'POST')
        
#         query.append(i)
    if any(i['auts_affils'][j]['islamic'] == True for j in i['auts_affils']):
        islamic.append(i)
    if any(i['auts_affils'][j]['country'] == 'US' for j in i['auts_affils']):
        us.append(i)
        
print('foreign: ', len(foreign))
print('islamic: ', len(islamic))
# # print(len(qs300))
# print(len(turkey)/len(foreign)*100)

In [ ]:
# exporting!

exp_fields = {
    'auts':         {'exp_value': 'Authors',                  'export': True, 'order': 1},
    'title':        {'exp_value': 'Title',                    'export': True, 'order': 2},
    'year':         {'exp_value': 'Year',                     'export': True, 'order': 3},
    'doc_type':     {'exp_value': 'Document Type',            'export': True, 'order': 4},
    'doi':          {'exp_value': 'DOI',                      'export': True, 'order': 5},
    'cites':        {'exp_value': 'Citations',                'export': True, 'order': 6},
    
    'journal':      {'exp_value': 'Journal',                  'export': True, 'order': 10},
    'publisher':    {'exp_value': 'Publisher',                'export': True, 'order': 11},
    'jcr_q1':       {'exp_value': 'JCR Q1 Journal',           'export': True, 'order': 12},
    'jcr_top1':     {'exp_value': 'JCR Top 1% Journal',       'export': True, 'order': 13},
    'a_hci':        {'exp_value': 'A_HCI',                    'export': True, 'order': 14},
    'esci':         {'exp_value': 'ESCI',                     'export': True, 'order': 15},
    'scie':         {'exp_value': 'SCIE',                     'export': True, 'order': 16},
    'ssci':         {'exp_value': 'SSCI',                     'export': True, 'order': 17},
    
    'auts_affils':  {'exp_value': 'Authors with Affiliation', 'export': True, 'order': 20},
    'depts':        {'exp_value': 'Unique Sharif Depts',      'export': True, 'order': 21},
    'has_foreign':  {'exp_value': 'Has Foreign Country',      'export': True, 'order': 22},
    'has_islamic':  {'exp_value': 'Has Islamic Country',      'export': True, 'order': 23},
    'countries':    {'exp_value': 'Unique Countries',         'export': True, 'order': 24},
    'institutions': {'exp_value': 'Institutions',             'export': True, 'order': 25},
    'has_qs100':    {'exp_value': 'Has QS100 Author',         'export': True, 'order': 26},
    'has_qs300':    {'exp_value': 'Has QS300 Author',         'export': True, 'order': 27},
    'sharif_only':  {'exp_value': 'Only Sharif Authors',      'export': True, 'order': 28},
}
exp_path = 'C:\\Users\\pmsoltani\\Desktop\\'
exp_name = 'export.txt'

exp_fields_keys = sorted(exp_fields.keys(), key = lambda x: exp_fields[x]['order'])
exp_headers = []
for item in exp_fields_keys:
    if exp_fields[item]['export']:
        exp_headers.append(exp_fields[item]['exp_value'])
exp_headers = '\t'.join(exp_headers) + '\n'

with io.open(os.path.join(exp_path, exp_name), 'w', encoding = "UTF-16") as tsvfile:
    tsvfile.write(exp_headers)
    for i in papers:
        # if any(i['auts_affils'][author]['Foreign'] == True for author in i['auts_affils']):
            
            auts = '; '.join([i['auts'][author] for author in i['auts']])
            auts_affils = '; '.join([i['auts_affils'][aut]['name_address'] for aut in i['auts_affils']])
            depts = '; '.join(list(set(i['depts']))) or '-'
            countries = '; '.join(list(set(i['countries']))) or '-'
            institutions = '; '.join(i['institutions'])

            exp_text = []
            for key in exp_fields_keys:
                if key == 'auts':
                    exp_text.append(auts)
                elif key == 'auts_affils':
                    exp_text.append(auts_affils)
                elif key == 'depts':
                    exp_text.append(depts)
                elif key == 'countries':
                    exp_text.append(countries)
                elif key == 'institutions':
                    exp_text.append(institutions)
                elif key in ['journal', 'jcr_q1', 'jcr_top1', 'a_hci', 'esci', 'scie', 'ssci']:
                    exp_text.append(str(i['src'][key]))
                else:
                    exp_text.append(str(i[key]))

            exp_text = '\t'.join(exp_text) + '\n'
            tsvfile.write(exp_text)